$\newcommand\reals{\mathbf{R}}$
$\newcommand\cd{\mathcal{D}}$
$\newcommand\ch{\mathcal{H}}$

# 2. Calculating Subgradients

## 2.1
Suppose $f_{1},\ldots,f_{m}:\reals^{d}\to\reals$
are convex functions, and 
\begin{align}
f(x)=\max_{i=1,\ldots,,m}f_{i}(x).
\end{align}
Let $k$ be any index for which $f_{k}(x)=f(x)$, and choose $g\in\partial f_{k}(x)$.
Show that $g\in\partial f(x)$.

#### Proof:
Since $g \in\partial f_k(x)$, we have,

\begin{align}
f_k(x+v) &\geq f_k(x) + g^Tv &&\forall v\in\reals^d
\end{align}

But 
\begin{align}
f(x) &= \max_{i=1,\ldots,,m}f_{i}(x)\\
&\geq f_{i}(x) && \forall i=1,\ldots,,m
\end{align}

Thus $\forall y\in \reals^d$, we have $f(y) \geq f_k(y)$

Now observe that
\begin{align}
f(x+v) &\geq f_k(x+v)\\
&\geq f_k(x) + g^Tv\\
&= f(x) + g^Tv && \text{since $f(x)=f_k(x)$}
\end{align}

Thus $g\in\partial f(x)$.

\newpage

## 2.2
Give a subgradient of $J(w)=\max\left\{ 0,1-yw^{T}x\right\} .$


From 2.1 we have

\begin{align}
\partial J(w) = \begin{cases}
-yx & yw^Tx \leq 1\\
0 & otherwise
\end{cases}
\end{align}

\newpage

# 3. Perceptron

## 3.1
Show that if $\left\{ x\mid w^{T}x=0\right\} $ is a separating hyperplane
for a training set $\cd=\left(\left(x_{1},y_{1}\right),\ldots,(x_{n},y_{n})\right)$,
then the average perceptron loss on $\cd$ is $0$. Thus any separating
hyperplane of $\cd$ is an empirical risk minimizer for perceptron
loss.

#### Proof

Assume $\{x\, \lvert\, w^Tx=0\}$ is a separating hyperplane. Then $y_iw^Tx_i > 0\,\, \forall\,\, i \in \{1, \ldots, n\}$.

Now observe that

\begin{align}
\text{Average perceptron loss} &= \frac{1}{n} \sum_{i=1}^n max\{0, -\hat{y_i}y_i\}\\
&= \frac{1}{n} \sum_{i=1}^n max\{0, -y_iw^Tx_i\}\\
&= \frac{1}{n} \sum_{i=1}^n 0 && \text{since $y_iw^Tx_i > 0$}\\
&= 0
\end{align}

\newpage

## 3.2
Let $\ch$ be the linear hypothesis space consisting of functions
$x\mapsto w^{T}x$. Consider running stochastic subgradient descent
(SSGD) to minimize the empirical risk with the perceptron loss. We'll
use the version of SSGD in which we cycle through the data points
in each epoch. Show that if we use a fixed step size $1$, we terminate
when our training data are separated, and we make the right choice
of subgradient,  then we are exactly doing the Perceptron algorithm.

By 2.1 we have $ \partial l(\hat{y},y) = \begin{cases}
-yx & yw^Tx < 0\\
0 & otherwise
\end{cases}$

With this choice of subgradient and a step size of 1 we can see that the update step is $w^{(k+1)} = \begin{cases}
w^{(k)} + y_ix_i & y_iw^Tx_i < 0\\
w^{(k)} & otherwise
\end{cases}$.

Note that this is precisely the update step in the perceptron algorithm.

Now for the terminating conditions.

Note that when the training data is separated, we have $y_iw^Tx_i > 0\,\, \forall\,\, i \in \{1, \ldots, n\}$. In the perceptron algorithm this will result in all_correct = True and termination of that algorithm. Thus the terminating conditions are equivalent.

\newpage

## 3.3
Suppose the perceptron algorithm returns $w$. Show that $w$ is a
linear combination of the input points. That is, we can write $w=\sum_{i=1}^{n}\alpha_{i}x_{i}$
for some $\alpha_{1},\ldots,\alpha_{n}\in\reals$. The $x_{i}$ for
which $\alpha_{i}\neq0$ are called support vectors. Give a characterization
of points that are support vectors and not support vectors.

From 3.2 we have that the update step is $w^{(k+1)} = \begin{cases}
w^{(k)} + y_ix_i & y_iw^Tx_i < 0\\
w^{(k)} & otherwise
\end{cases}$.

Then since $w^{(0)}=0$ and $y_i \in \{-1,1\}$ it follows that can write $w=\sum_{i=1}^{n}\alpha_{i}x_{i}$
for some $\alpha_{1},\ldots,\alpha_{n}\in\reals$.

The support vectors are vectors that were misclassified at some point during the optimization. When this happens, $y_iw^Tx_i < 0$ which results in $w^{(k+1)} = w^{(k)} + y_ix_i$ and so $\alpha_{i}\neq0$.

\newpage

# 4. The Data

## 4.1
Load all the data and randomly split it into 1500 training examples and 500 validation examples.

In [9]:
from load import shuffle_data
from sklearn.model_selection import train_test_split

# I added a return statement to shuffle_data
data = shuffle_data()
train, val = train_test_split(data, train_size=1500, test_size=500)

\newpage

# 5. Sparse Representations

## 5.1
Write a function that converts an example (e.g. a list of words) into a sparse bag-of-words
representation.

In [13]:
from collections import Counter

def to_sparse(word_list):
    return Counter(word_list)

\newpage

# 6. Support Vector Machine via Pegasos

## 6.1
Consider the ``stochastic'' SVM objective function,
which is the SVM objective function with a single training point\footnote{Recall that if $i$ is selected uniformly from the set $\left\{ 1,\ldots,m\right\} $,
then this stochastic objective function has the same expected value
as the full SVM objective function.}: $J_{i}(w)=\frac{\lambda}{2}\|w\|^{2}+\max\left\{ 0,1-y_{i}w^{T}x_{i}\right\} $.
The function $J_{i}(\theta)$ is not differentiable everywhere. Give
an expression for the gradient of $J_{i}(w)$ where it's defined,
and specify where it is not defined.

#### Solution
$\nabla J_i(w) = \begin{cases}
\lambda w -y_ix_i & y_iw^Tx_i < 1\\
\lambda w & y_iw^Tx_i > 1\\
undefined & y_iw^Tx_i = 1
\end{cases}$

\newpage

## 6.2 
Show that a subgradient of $J_{i}(w)$ is given by 
\begin{eqnarray*}
g & = & \begin{cases}
\lambda w-y_{i}x_{i} & \mbox{for }y_{i}w^{T}x_{i}<1\\
\lambda w & \mbox{for }y_{i}w^{T}x_{i}\ge1.
\end{cases}
\end{eqnarray*}
You may use the following facts without proof: 1) If $f_{1},\ldots,f_{m}:\reals^{d}\to\reals$
are convex functions and $f=f_{1}+\cdots+f_{m}$, then $\partial f(x)=\partial f_{1}(x)+\cdots+\partial f_{m}(x)$.
2) For $\alpha\ge0$, $\partial\left(\alpha f\right)(x)=\alpha\partial f(x)$.

Let $ f_1(w) = \frac{\lambda}{2} \Vert w\Vert^2$ and $ f_2(w) = max \{0,1-y_iw^Tx_i\}$. Note that $J_i(w) = f_1(w) + f_2(w)$.

Now $\partial f_1(w) = \lambda w$.

From the results of question 2.1 we also have $ \partial f_2(w) = \begin{cases}
-y_{i}x_{i} & \mbox{for }y_{i}w^{T}x_{i}<1\\
0 &\mbox{for }y_{i}w^{T}x_{i}\ge1.
\end{cases}$

Then from fact 1 we can conclude that $g = \begin{cases}
\lambda w-y_{i}x_{i} & \mbox{for }y_{i}w^{T}x_{i}<1\\
\lambda w & \mbox{for }y_{i}w^{T}x_{i}\ge1.
\end{cases}$





\newpage